## Quaternary Heusler Query from OQMD database by mysql code

1.Use mysql code to find compound (4 atoms, 4 elements ,spacegroup 216, convex<0.1, formation <0)
attention: may have duplicated entry_id

In [ ]:
select * from (select * from formation_energies where stability < 0.1 and delta_e <0) as t1 inner join (select * from structures where natoms=4 and ntypes=4 and spacegroup_id=216 and label='static') as t2 inner join (select * from calculations where label='static') as t3 where t1.entry_id =t2.entry_id and t2.entry_id = t3.entry_id into outfile './full_QH_magmom.csv' FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\r\n';

2.Use qmpy.structure to find compounds structures with the entry_id.

In [ ]:
from qmpy import *
import pandas as pd
from collections import OrderedDict
import json
entry = []
with open ("entry_id.txt",'r') as f:
    data = f.readlines()
    for i in data:
        entry.append(i.rstrip('\n'))

QH = OrderedDict()
for i in entry:
    s=Structure.objects.filter(entry_id = i,label='static')[0]
    QH[i] = {"composition":s.composition_id,"entry_id":i,"structure":io.poscar.write(s)}

with open("QH_mysql.json","w") as ff:
    json.dump(QH,ff)

3.post processing data: (Query_QH_mysql-magmom)
key functions:

In [ ]:
### compare poscar of two structures
from pymatgen.io.vasp import Poscar
from pymatgen.analysis.structure_matcher import StructureMatcher

def compare_poscar(poscar1,poscar2):
    sm = StructureMatcher()
    poscar1 = Poscar.from_string(poscar1)
    poscar2 = Poscar.from_string(poscar2)
    s1 = poscar1.structure
    s2 = poscar2.structure
    similar = sm.fit(s1,s2)
    return similar

In [ ]:
#get QH XX'YZ correct name
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import IStructure
def get_correct_name(j):
    X1 = None
    X2 = None
    Y = None
    Z = None
    poscar = QH[j]['poscar']
    primitive_structure = IStructure.from_str(poscar, fmt='poscar')
    SGA = SpacegroupAnalyzer(primitive_structure)
    conventional_structure = SGA.get_conventional_standard_structure()
    for i in range(0,16):
        coord = conventional_structure.sites[i]
        if coord.a == coord.b and coord.b == coord.c and coord.c==0.5:
            for ele,num in conventional_structure.sites[i].species.items():
                Z = ele.symbol
        if coord.a == coord.b and coord.b == coord.c and coord.c==0:
            for ele,num in conventional_structure.sites[i].species.items():
                Y = ele.symbol
        if coord.a == coord.b and coord.b == coord.c and coord.c==0.25:
            for ele,num in conventional_structure.sites[i].species.items():
                X2 = ele.symbol
        if coord.a == 0.75 and coord.b == coord.c and coord.c==0.25:
            for ele,num in conventional_structure.sites[i].species.items():
                X1 = ele.symbol
    name = 'POSCAR_'+X1+X2+Y+Z
    conventional_structure.to(fmt="poscar",filename='./OQMD_all/%s'%name)
    return name,conventional_structure.lattice.abc[0],conventional_structure.lattice.abc[1],conventional_structure.lattice.abc[2]

In [ ]:
## remove rare-earth compounds
from pymatgen.core import Composition, Element
sb_uni_no_rare = deepcopy(sb_unique)
for i in sb_unique:
    Comp = Composition(i.split('_')[0])
    for ele,num in Comp.items():
        if ele.is_rare_earth_metal == True:
            sb_uni_no_rare.remove(i)
            break